# Tutorial

This short tutorial will teach you how to consume senpy services for several tasks, and how to take advantage of the features of the framework.

In particular, it covers:

* Annotating text with sentiment and emotion using interoperable services
* Switching to different services (service interoperability)
* Getting results in different formats (Turtle, XML, text...)
* Asking for specific emotion models (automatic model conversion)

The tutorial is originally published as an interactive [jupyter notebook](https://jupyter.org/) (`ipynb` file) that you can download and run locally.
The on-line documentation contains a **static** version of the notebook.

## Requirements

We will use the demo server at http://senpy.gsi.upm.es and the requests library.

We will use a variable for our endpoint.
To try these examples on other instances simply change the value of this variable and re-run the query:

In [1]:
endpoint = 'http://senpy.gsi.upm.es/api'

We will also add a little helper function (`query`) to simplify our queries and pretty-print the results with syntax highlighting:

In [2]:
import requests
from IPython.display import Code
     
def query(endpoint, **kwargs):
    '''Query a given Senpy endpoint with specific parameters, and prettify the output'''
    res = requests.get(endpoint,
                       params=kwargs)
    if res.status_code != 200:
        raise Exception(res)
    return Code(res.text, language=kwargs.get('outformat', 'json-ld'))

## Sentiment Analysis of Text

To start, let us analyse the sentiment in the following sentence: *senpy is awesome*.

For now, we will use the [sentiment140](http://www.sentiment140.com/) service, through the sentiment140 plugin.
We will later cover how to use a different service.


In [3]:
query(f'{endpoint}/sentiment140', input="Senpy is awesome")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudDE0MD9pbnB1dD1TZW5weStpcythd2Vzb21lIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [
        {
          "@type": "Sentiment",
          "marl:hasPolarity": "marl:Positive",
          "prov:wasGeneratedBy": "prefix:Analysis_1563372853.439696"
        }
      ],
      "nif:isString": "Senpy is awesome",
      "onyx:hasEmotionSet": []
    }
  ]
}

Senpy services always return an object of type `senpy:Results`, with a list of entries.
You can think of an entry as a self-contained textual context (`nif:Context` and `senpy:Entry`).
Entries can be as short as a sentence, or as long as a news article.

Each entry has a `nif:isString` property that contains the original text of the entry, and several other properties that are provided by the plugins.

For instance, sentiment annotations are provided through `marl:hasOpinion`.

The annotations are semantic.
This is clear if we request a different semantic format, such as `turtle`.
The output format is controlled with the `outformat` parameter:

In [4]:
query(f'{endpoint}/sentiment140',
      input="Senpy is awesome",
      outformat="turtle")

@prefix : <http://www.gsi.upm.es/onto/senpy/ns#> .
@prefix dc: <http://dublincore.org/2012/06/14/dcelements#> .
@prefix emoml: <http://www.gsi.upm.es/ontologies/onyx/vocabularies/emotionml/ns#> .
@prefix endpoint: <http://senpy.gsi.upm.es/api/> .
@prefix fam: <http://vocab.fusepool.info/fam#> .
@prefix marl: <http://www.gsi.upm.es/ontologies/marl/ns#> .
@prefix nif: <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#> .
@prefix onyx: <http://www.gsi.upm.es/ontologies/onyx/ns#> .
@prefix prefix: <http://senpy.invalid/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix senpy: <http://www.gsi.upm.es/onto/senpy/ns#> .
@prefix wna: <http://www.gsi.upm.es/ontologies/wnaffect/ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

prefix: a senpy:Entry ;
    nif:isString "Senpy is awesome" ;
    marl:hasOpinion [ a senpy:Sentiment ;
            marl:hasPolarity "marl:Positive" ;
            prov:wasGeneratedBy prefix:Analysis_1563372853.6874764 ] .

[] a senpy:Results ;
    prov:used prefix: .

## Moving to a different service

All senpy plugins use the same API, which makes moving from one service to another a breeze.

Let us modify the earlier example, which uses the `sentiment140` service, to use a different service (e.g. the `sentiment-basic` plugin).
We can do it just by changing the URL of the service:

In [5]:
query(f'{endpoint}/sentiment-basic',
      input="Senpy is awesome")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudC1iYXNpYz9pbnB1dD1TZW5weStpcythd2Vzb21lIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [
        {
          "@type": "Sentiment",
          "marl:hasPolarity": "marl:Neutral",
          "prov:wasGeneratedBy": "prefix:Analysis_1563372853.8034046"
        }
      ],
      "nif:isString": "Senpy is awesome",
      "onyx:hasEmotionSet": []
    }
  ]
}

As you can see, the structure and annotation schema of the response is the same.
This makes it very easy to compare and migrate to different services.

Service interoperability is not only useful for users.
It is also key for other features such as [automated evaluation](http://senpy.readthedocs.io/Evaluation.html).
This is a compelling reason to adapt existing services to use the Senpy API.
In fact, the `sentiment140` senpy service is proxy to the public [Sentiment 140 service](http://www.sentiment140.com/).

## Emotion analysis


Senpy uses the `onyx` vocabulary to represent emotions, which incorporates the notion of `EmotionSet`'s, an emotion that is composed of several emotions.
In a nutshell, an `Entry` is linked to one or more `EmotionSet`, which in turn is made up of one or more `Emotion`.

Let's illustrate it with an example, using the `emotion-depechemood` plugin.

In [6]:
query(f'{endpoint}/emotion-depechemood',
      input="Senpy is a wonderful that service")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tZGVwZWNoZW1vb2Q_aW5wdXQ9U2VucHkraXMrYSt3b25kZXJmdWwrdGhhdCtzZXJ2aWNlIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful that service",
      "onyx:hasEmotionSet": [
        {
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:negative-fear",
              "onyx:hasEmotionIntensity": 0.06258366271018097
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:amusement",
              "onyx:hasEmotionIntensity": 0.15784834034155437
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:anger",
              "onyx:hasEmotionIntensity": 0.08728815135373413
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:annoyance",
              "onyx:hasEmotionIntensity": 0.12184635680460143
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:indifference",
              "onyx:hasEmotionIntensity": 0.1374081151031531
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:joy",
              "onyx:hasEmotionIntensity": 0.12267040802346799
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:awe",
              "onyx:hasEmotionIntensity": 0.21085262130713067
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:sadness",
              "onyx:hasEmotionIntensity": 0.09950234435617733
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1563372853.9469151"
        }
      ]
    }
  ]
}

As you have probably noticed, there are several emotions in this result, each with a different intensity.

We can also tell senpy to only return the emotion with the maximum intensity using the `maxemotion` parameter:

In [7]:
query(f'{endpoint}/emotion-depechemood',
      input="Senpy is a wonderful service",
      maxemotion=True)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tZGVwZWNoZW1vb2Q_aW5wdXQ9U2VucHkraXMrYSt3b25kZXJmdWwrc2VydmljZSZtYXhlbW90aW9uPVRydWUj",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful service",
      "onyx:hasEmotionSet": [
        {
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:awe",
              "onyx:hasEmotionIntensity": 0.21085262130713067
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1563372854.0490181"
        }
      ]
    }
  ]
}

## Emotion conversion

Sometimes the model used by a plugin is not right for your application. Senpy ships with emotion conversion capabilities: you can ask for a specific emotion model in your request and the service will try to automatically convert the results.

For example, the `emotion-anew` plugin uses the dimensional `pad` (or VAD, valence-arousal-dominance) model, as we can see here:

In [8]:
query(f'{endpoint}/emotion-anew',
      input="Senpy is a wonderful service and I love it")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tYW5ldz9pbnB1dD1TZW5weStpcythK3dvbmRlcmZ1bCtzZXJ2aWNlK2FuZCtJK2xvdmUraXQj",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful service and I love it",
      "onyx:hasEmotionSet": [
        {
          "@id": "Emotions0",
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@id": "Emotion0",
              "@type": "Emotion",
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#arousal": 6.44,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#dominance": 7.11,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#valence": 8.72,
              "prov:wasGeneratedBy": "prefix:Analysis_1563372854.2822595"
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1563372854.2822595"
        }
      ]
    }
  ]
}

If we need a category level, we can ask for the equivalent results in the `big6` model:

In [9]:
query(f'{endpoint}/emotion-anew',
      input="Senpy is a wonderful service and I love it",
      emotionmodel="emoml:big6")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tYW5ldz9pbnB1dD1TZW5weStpcythK3dvbmRlcmZ1bCtzZXJ2aWNlK2FuZCtJK2xvdmUraXQmZW1vdGlvbm1vZGVsPWVtb21sJTNBYmlnNiM%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful service and I love it",
      "onyx:hasEmotionSet": [
        {
          "@id": "Emotions0",
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@id": "Emotion0",
              "@type": "Emotion",
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#arousal": 6.44,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#dominance": 7.11,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#valence": 8.72,
              "prov:wasGeneratedBy": "prefix:Analysis_1563372854.3354168"
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1563372854.3354168"
        }
      ]
    }
  ]
}

Because we don't usually care about the original emotion, the conversion can be presented in three ways:

* full: the original and converted emotions are included at the same level
* filtered: the original emotion is replaced by the converted emotion
* nested: the original emotion is replaced, but the converted emotion points to it

For example, here's how the `nested` structure would look like:

In [10]:
query(f'{endpoint}/emotion-anew',
      input="Senpy is a wonderful service and I love it",
      emotionmodel="emoml:big6",
      conversion="nested")

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tYW5ldz9pbnB1dD1TZW5weStpcythK3dvbmRlcmZ1bCtzZXJ2aWNlK2FuZCtJK2xvdmUraXQmZW1vdGlvbm1vZGVsPWVtb21sJTNBYmlnNiZjb252ZXJzaW9uPW5lc3RlZCM%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful service and I love it",
      "onyx:hasEmotionSet": [
        {
          "@id": "Emotions0",
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@id": "Emotion0",
              "@type": "Emotion",
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#arousal": 6.44,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#dominance": 7.11,
              "http://www.gsi.upm.es/ontologies/onyx/vocabularies/anew/ns#valence": 8.72,
              "prov:wasGeneratedBy": "prefix:Analysis_1563372854.3876536"
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1563372854.3876536"
        }
      ]
    }
  ]
}

## Learn more

[A separate notebook](Advanced.ipynb) covers advanced topics such as listing all plugins available in an endpoint, or creating analysis pipelines that chain several analysis services.